In [11]:
import requests
from bs4 import BeautifulSoup
import csv
from elasticsearch import Elasticsearch, helpers
import configparser
import pandas as pd

config = configparser.ConfigParser()
config.read('example.ini')

es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

# 상명대학교 전자도서관 베스트 셀러 URL
URL="https://libebook.smu.ac.kr/FxLibrary/product/list/?category_type=book&cateopt=best" 

# 총 페이지 수 추출 함수
def get_page_count():
    result = requests.get(URL)

    soup = BeautifulSoup(result.text, "html.parser")

    count = len(soup.select("div.paging span a"))+1
    
    return count

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    book_list = []
    
    for page in range(page_count):
    
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://libebook.smu.ac.kr/FxLibrary/product/list/?itemdv=1&sort=3&page={page+1}&itemCount=20&pageCount=10&category=&middlecategory=&cateopt=best&group_num=recommand&catenavi=main&category_type=book&searchoption=&keyoption=&keyoption2=&keyword=&listfilter=all_list&selectview=list_on&searchType=&name=&publisher=&author=&terminal=")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.select("div.book_list ul.list.typelist li.item")
        
        for rs in results:
            
            try:
                subject=rs.select_one("div.subject a").text
                
                category=rs.select_one("div.category span").text
    
                writer=rs.select("div.info ul.i1 a")[0].text

                publisher=rs.select("div.info ul.i1 a")[1].text

                date=rs.select("div.info ul.i1 li")[2].text
                
                if date=='0000-00-00' or "NaN":
                    
                    date='1970-01-01'

                checkout=rs.select("div.info ul.i2 li")[2].text

                like=rs.select("div.info ul.i2 li")[3].text
            
                book_list.append([subject, category, writer, publisher, date, checkout, like])
            
                es.index(
                    index="sm_all_100",
                    document={
                        '제목':subject,
                        'genre':category,
                        'writer':writer,
                        'publisher':publisher,
                        'date':date,
                        'chechout':checkout,
                        'like':like
                    })
                
            except Exception as e:    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
                print('예외가 발생했습니다.', e)
                
            
    return book_list

# CSV 저장
page_count=get_page_count()

books=extract_books(page_count)

df = pd.DataFrame(books, columns = ["제목", "genre", "writer", "pulisher", "date", "checkout","like"])

df.to_csv("sm_all_100_1", index = False, encoding = 'utf-8-sig')

#file=open("sm_best_100_1.csv",mode="w")

#write=csv.writer(file)

#write.writerow(["category", "subject", "writer", "publisher","date","checkout","like"])

#for book in books:
    
#    write.writerow(list(book.values()))

C:\Users\shjo\AppData\Local\Temp\ipykernel_9324\422925989.py:11: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Scrapping page 1
Scrapping page 2
Scrapping page 3
예외가 발생했습니다. BadRequestError(400, 'mapper_parsing_exception', "failed to parse field [date] of type [date] in document with id 'D5YNdIYBYveY2p9AAVgG'. Preview of field's value: '0000-00-00'")
Scrapping page 4
Scrapping page 5


In [18]:
df

,제목,genre,writer,pulisher,date,checkout,like
0,데미안,[문학],헤르만 헤세,더클래식,2015-07-23,누적대출 3189,추천 6
1,이방인,[문학],알베르 카뮈,주식회사 자화상,2019-03-14,누적대출 2019,추천 2
2,무례한 사람에게 웃으며 대처하는 법,[에세이/산문],정문정,가나출판사,2018-01-08,누적대출 1755,추천 5
3,바깥은 여름,[문학],김애란,문학동네,2017-08-09,누적대출 1284,추천 10
4,11분,[문학],파울로 코엘료,문학동네,2011-04-29,누적대출 1140,추천 0
...,...,...,...,...,...,...,...
95,무엇이 우리의 생각을 지배하는가,[인문],엘든 테일러,랜덤하우스코리아,2012-03-17,누적대출 209,추천 0
96,Dr. 정혜신의 셀프 피부 관리법,[가정/생활],정혜신,경향미디어,2009-08-14,누적대출 206,추천 0
97,나비와 뼈다귀,[장르문학],조효은,로맨스토리,2012-02-24,누적대출 206,추천 0
98,작별하지 않는다,[문학],한강,문학동네,2021-09-27,누적대출 204,추천 0
